the faces' latent space is created with pbaylies' StyleGAN encoder (building on original work from Puzer): 

pbaylies repo: https://github.com/pbaylies/stylegan-encoder
StyleGAN paper: https://arxiv.org/abs/1812.04948

once run as https://colab.research.google.com/github/sifbuilder/eodoes/blob/master/packages/eodoes-do-insights/code/randfaces.ipynb



In [0]:
# will define globals

cnnid = 'randfaces'

autoDownloadImages = 1 # auto run mode - get uri[uri.rfind("/")+1:] files from imgsToAutodownload
uploadRawImages = 0 # manually upload raw images
takePhotoImages = 0 # take photo for raw images

imgsToAutodownload = [
  "https://github.com/sifbuilder/eodoes/raw/master/packages/eodoes-eodo-eofaces/img/amgc.jpg"
]

downloadResults = 0 # download results

emptyspace = 0 # or reset runtime
toSmptyFolders = ["stylegan-encoder", "InterFaceGAN", "output_vectors.npy"]

usegdrive = 0

In [0]:
# process

import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import humanize
import psutil
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
printm() 

In [0]:
!nvcc --version

In [0]:
# runtime type must be GPU enabled

import os
import pickle
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import PIL.Image

import tensorflow as tf
tf.VERSION
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!pwd

In [0]:
# would clean root folder

if emptyspace == 1:
  print("remove folders %s" %toSmptyFolders)
  for folder in toSmptyFolders:
    print("folder: %s" %folder)
    !rm -r $folder

In [0]:
# stylegan folder signature

signature='encode_images.py'
isExist = os.path.exists(signature)
if isExist:
  print("in stylegan. up to root")
  %cd ..

#### pbaylies stylegan model

In [0]:
# clone pbaylies stylegan encoder:

path = 'stylegan-encoder'
isExist = os.path.exists(path)
if not isExist:
  print("stylegan-encoder does not exist. will clone")
  !git clone https://github.com/pbaylies/stylegan-encoder
else:
  print("stylegan-encoder already exists. will not clone")

In [0]:
# cd into the encoder repo folder, then create image folders

if os.path.exists('stylegan-encoder') is True:
    os.chdir('stylegan-encoder')
if os.path.exists('aligned_images') is False:
    os.mkdir('aligned_images')
if os.path.exists('raw_images') is False:
    os.mkdir('raw_images')

In [0]:
import dnnlib
import dnnlib.tflib as tflib
import config
from tqdm import tqdm_notebook as tqdm
tflib.init_tf()

#### pretrained encoder

In [0]:
# pretrained resnet encoder - take image as input and estimate its latent code

!gdown https://drive.google.com/uc?id=1aT59NFy9-bNyXjDuZOTMl0qX0jmZc6Zb
!mkdir -p data
!mv finetuned_resnet.h5 data
!rm -rf generated_images latent_representations

In [0]:
print("aligned_images contains %d images ready for encoding!" %len(os.listdir('aligned_images/')))

#### nvidia trained network

In [0]:
# download the StyleGAN network from NVIDIA trained on faces 
# karras2019stylegan-ffhq-1024x1024.pkl
!gdown https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ

In [0]:
# load the stylegan and instantiate the generator

fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

# bsize = 8
bsize = 1
synthesis_kwargs = dict(output_transform=fmt, minibatch_size=bsize) 

model_dir = './'
model_path = 'karras2019stylegan-ffhq-1024x1024.pkl'
print("Loading StyleGAN model from %s..." %model_path)

with dnnlib.util.open_url(model_path) as f:
  generator_network, discriminator_network, averaged_generator_network = pickle.load(f)
  _G = generator_network
  _D = discriminator_network
  Gs = averaged_generator_network
  
print("StyleGAN loaded")

In [0]:
# generator_network.print_layers() # show generator model

### generate_images(generator, latents)

In [0]:
# generate and plot random images



def generate_images(generator, latent_vector, z = True):
  latent_dim = 512
  batch_size = latent_vector.shape[0]
  
  if z: #Start from z: run the full generator network
    latents = latent_vector.reshape((batch_size, latent_dim))
    return generator.run(latents, None, randomize_noise=True, **synthesis_kwargs)
  else: #Start from w: skip the mapping network
    latents = latent_vector.reshape((batch_size, 18, latent_dim))
    return generator.components.synthesis.run(latents, randomize_noise=True, **synthesis_kwargs)


### plot_images (Gs) ... generate_images

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_imgs(model, rows, columns):
  rndvects = {}
  for i in range(rows):
    f, axarr = plt.subplots(1, columns, figsize = (10, 8))

    for j in range(columns):

      latent_dim = 512
      rndvects[i,j] = np.random.randn(1, latent_dim ) # [[ 0.55756274 ... ]]

      latent_vector = rndvects[i,j]
      img = generate_images(model, latent_vector, z = True)[0]
           
      axarr[j].imshow(img)
      axarr[j].axis('off')
      axarr[j].set_title('Resolution: %s' %str(img.shape))
    plt.show()

import numpy as np
plot_imgs(Gs, 3, 3)

### show_faves (Gs)

In [0]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

def bookmark(latents, new_faves):
    for f in new_faves:
        faves.append(latents[f])

def show_faves(faves):
    latents = np.array(faves)
    labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
    n = len(faves)
    nr, nc = math.ceil(n / 6), 6
    for r in range(nr):

        images = Gs.run(latents[6*r:min(n-1, 6*(r+1))], 
                        None, 
                        truncation_psi=0.5, 
                        randomize_noise=False, 
                        output_transform=fmt)

        img1 = np.concatenate([img for img in images], axis=1)
        plt.figure(figsize=(24,4))
        plt.imshow(img1)

### get latent interpolations

In [0]:
def get_latent_interpolation(endpoints, num_frames_per, mode, shuffle):
    if shuffle:
        random.shuffle(endpoints)
    num_endpoints, dim = len(endpoints), len(endpoints[0])
    num_frames = num_frames_per * num_endpoints
    endpoints = np.array(endpoints)
    latents = np.zeros((num_frames, dim))
    for e in range(num_endpoints):
        e1, e2 = e, (e+1)%num_endpoints
        for t in range(num_frames_per):
            frame = e * num_frames_per + t
            r = 0.5 - 0.5 * np.cos(np.pi*t/(num_frames_per-1)) if mode == 'ease' else float(t) / num_frames_per
            latents[frame, :] = (1.0-r) * endpoints[e1,:] + r * endpoints[e2,:]
    return latents

def get_latent_interpolation_bspline(endpoints, nf, k, s, shuffle):
    if shuffle:
        random.shuffle(endpoints)
    x = np.array(endpoints)
    x = np.append(x, x[0,:].reshape(1, x.shape[1]), axis=0)
    nd = x.shape[1]
    latents = np.zeros((nd, nf))
    nss = list(range(1, 10)) + [10]*(nd-19) + list(range(10,0,-1))
    for i in tqdm(range(nd-9)):
        idx = list(range(i,i+10))
        tck, u = interpolate.splprep([x[:,j] for j in range(i,i+10)], k=k, s=s)
        out = interpolate.splev(np.linspace(0, 1, num=nf, endpoint=True), tck)
        latents[i:i+10,:] += np.array(out)
    latents = latents / np.array(nss).reshape((512,1))
    return latents.T

### gen_imgs (Gs)

In [0]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

def gen_imgs(latents, labels):
  batch_size = 8
  num_frames = latents.shape[0]
  num_batches = int(np.ceil(num_frames/batch_size))
  images = []
  for b in tqdm(range(num_batches)):

    latent_vector = latents[b*batch_size:min((b+1)*batch_size, num_frames-1), :]
    new_images = Gs.run(latent_vector, 
                        None, 
                        truncation_psi=truncation, 
                        randomize_noise=False, 
                        output_transform=fmt)

    for img in new_images:
      images.append(img)
  return images

### make_movie ... gen_imgs (Gs)

In [0]:
def make_movie(images, out_dir, out_name):
    temp_dir = 'frames%06d'%int(1000000*random.random())
    os.system('mkdir %s'%temp_dir)
    for idx in tqdm(range(len(images))):
        PIL.Image.fromarray(images[idx], 'RGB').save('%s/frame%05d.png' % (temp_dir, idx))
    cmd = 'ffmpeg -i %s/frame%%05d.png -c:v libx264 -pix_fmt yuv420p %s/%s.mp4' % (temp_dir, out_dir, out_name)
    print(cmd)
    os.system(cmd)
    os.system('rm -rf %s'%temp_dir)

### random_sample (Gs)

In [0]:
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

truncation = 0.5 # default truncation_psi

# def random_sample(num_images, scale):
#     latents = np.random.RandomState(int(1000*random.random())).randn(num_images, *Gs.input_shapes[0][1:])
#     labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
#     images = Gs.run(latents, None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
#     images_ct = np.concatenate([img for img in images], axis=1)
#     plt.figure(figsize=(scale*num_images, scale))
#     plt.imshow(images_ct)
#     return images, latents
    
def random_sample(num_images, scale):
  latents = np.random.RandomState(int(1000*random.random())).randn(num_images, *Gs.input_shapes[0][1:])
  labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])

  images = Gs.run(latents, 
                  None, 
                  truncation_psi=truncation, 
                  randomize_noise=False, 
                  output_transform=fmt)

  images_ct = np.concatenate([img for img in images], axis=1)
  plt.figure(figsize=(scale*num_images, scale))
  plt.imshow(images_ct)
  plt.axis('off')
  plt.savefig('download.png')
  return images, latents


### create_image

In [0]:
# https://github.com/pearsonkyle/Artificial-Art/blob/master/animated_mosaic.py

import matplotlib.pyplot as plt
import numpy as np

def create_image(gen_imgs, name, xsize=4, ysize=4):
    fig, axs = plt.subplots(xsize, ysize, figsize=(xsize*2,ysize*2))
    plt.subplots_adjust(left=0.05,bottom=0.05,right=0.95,top=0.95, wspace=0.2, hspace=0.2)

    cnt = 0
    for i in range(ysize):
        for j in range(xsize):
            imgs = gen_imgs[cnt]
            axs[i,j].imshow(imgs)
            axs[i,j].axis('off')
            cnt += 1

    fig.savefig(name, facecolor='white' )
    
    plt.close()

### make video ... gen_imgs ... random_sample

In [0]:
# get latent interpolations, then generate images

imgnb = 5 # images in row
scale = 2
images, latents = random_sample(imgnb, scale)

latents = get_latent_interpolation(latents, 30, 'linear', False)
labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])

images = gen_imgs(latents, labels)    

make_movie(images, '.', 'faves13')

### display video

In [0]:
# display videos

from moviepy.editor import *
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.Clip import Clip
from IPython.display import display

vid_path = './faves13.mp4'
print ("vid_path %s" %vid_path)
clip = VideoFileClip(vid_path)
display(clip.ipython_display(height=512, autoplay=1, loop=1))


### animate mosaic

In [0]:
# https://github.com/pearsonkyle/Artificial-Art/blob/master/animated_mosaic.py

print("cwd", os.getcwd()) # assume in stylegan-endoder
if os.path.exists('out_images') is False:
    os.mkdir('out_images')   

img_rows = 128
img_cols = 128
channels = 3
latent_dim = 512 # 256
name = 'goodsell_256_128'

# make sure to load in the correct sized data
# dcgan = DCGAN(img_rows, img_cols, channels, latent_dim, name)
# dcgan.load_weights(generator_file="generator ({})4000.h5".format(dcgan.name), 
#                    discriminator_file="discriminator ({}).h5".format(dcgan.name))
# predictor = dcgan.generator.predict(img_vector)
def predictor(latents):
  img_array = generate_images(gs, latents)
  return img_array

# starting point for every image
seed_start = np.random.normal(0, 1, (16, latent_dim))

# these parameters will change every time step
latentSpeed = np.random.normal(3, 1, (16, latent_dim))
vary = np.copy(seed_start)

# interpolate on time
def interp(s, t, T, v):
  return s + np.sin( 2*np.pi*(t/T) * v ) 

# video settings
time = 0
fps = 30
maxTime = 5 # seconds
frameCount = 0

while (time <= maxTime):

    # for each image
    for i in range(len(seed_start)): 
        
      # change the latent variables
      for j in range(latent_dim):
        s = seed_start[i][j]
        v = latentSpeed[i][j]
        vary[i][j] = interp(s, time, maxTime, v)
        # vary[i][j] = seed_start[i][j] + np.sin( 2*np.pi*(time/maxTime) * latentSpeed[i][j] ) 
        # print("vary i j", vary[i][j]) # -0.3797196489613839
        # vector = generate_images(Gs,  vary[i][j])

    # gen_imgs = predictor(vary) # dcgan.generator.predict
    gen_imgs = generate_images(Gs, vary) # cannot reshape array of size 4096 into shape (16,512)

    create_image(gen_imgs, "out_images/favimgs_{0:05d}.png".format(frameCount)  )

    frameCount += 1
    time += 1./fps

# ffmpeg -framerate 30 -i "galaxy_%05d.png" -i "Khruangbin_Friday Morning.mp3" -map 0:v:0 -map 1:a:0 -shortest -c:v libx264 -pix_fmt yuv420p -strict -2 galaxy.mp4 
# ffmpeg -framerate 30 -i "nebula_%05d.png" -i "planet_caravan.mp3" -map 0:v:0 -map 1:a:0 -c:v libx264 -pix_fmt yuv420p nebula.mp4
# ffmpeg -framerate 30 -i "fluidart_%05d.png" -c:v libx264 -pix_fmt yuv420p fluidart.mp4    

# ffmpeg -i frames953417/frame%05d.png -c:v libx264 -pix_fmt yuv420p ./faves13.mp4

In [0]:
from moviepy.editor import *
import cv2

out_path = 'output_videos/'
os.makedirs(out_path, exist_ok=True)

image_folder = "out_images/"
print ("images folder %s" %image_folder)
video_fps = 30.


images = [img_path for img_path in sorted(os.listdir(image_folder)) if '.png' in img_path]
os.makedirs(out_path, exist_ok=True)

prev_id = None
img_sets = []
for img_path in images:
  img_id = img_path.split('_')[0]
  if img_id == prev_id: #append
    img_sets[-1].append(img_path)
    
  else: #start a new img set
    img_sets.append([])
    img_sets[-1].append(img_path)
  prev_id = img_id

print("Found %d image sets!\n" %len(img_sets))
if image_folder[-1] != '/':
  image_folder += '/'
 

# make video per latent direction
for i in range(len(img_sets)):
  print("Generating video %d..." %i)
  set_images = []
  vid_name = out_path + 'out_video_%02d' %(i)
  print("make video: %s" %vid_name)
  for img_path in img_sets[i]:
    img_fullpath = image_folder + img_path

    frame = cv2.imread(img_fullpath)
    set_images.append(frame)

  set_images.extend(reversed(set_images))
  make_movie(set_images, ".", vid_name)
  

In [0]:
vids = sorted([f for f in os.listdir('output_videos')])
print ("vids %s" %vids)
for i, vid_name in enumerate(vids):
  vid_path = 'output_videos/%s' %vid_name
  print ("vid_path %s" %vid_path)
  clip = VideoFileClip(vid_path)
  display(clip.ipython_display(height=512, autoplay=1, loop=1))

In [0]:
for i, vid_name in enumerate(vids):
  vid_path = 'output_videos/%s' %vid_name  
  print("vid_path: %s" %vid_path)  
  files.download(vid_path) 